<a href="https://colab.research.google.com/github/iambaangkok/261459-Deep-Learning/blob/master/Identification_of_Surface_Cracks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Download dataset from kaggle

In [4]:
! pip install -q kaggle

In [5]:
%reset -f

from google.colab import files

In [6]:
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"iambaangkok","key":"4dc0b759a44f63015879dfef976d16d1"}'}

In [7]:
! mkdir ~/.kaggle

In [8]:
! cp kaggle.json ~/.kaggle/

In [9]:
! chmod 600 ~/.kaggle/kaggle.json

In [10]:
# ! kaggle datasets list

In [11]:
! kaggle datasets download -d arunrk7/surface-crack-detection

100% 232M/233M [00:01<00:00, 222MB/s]
100% 233M/233M [00:01<00:00, 216MB/s]


In [12]:
! mkdir dataset

In [13]:
! unzip surface-crack-detection.zip -d dataset

Streaming output truncated to the last 5000 lines.
  inflating: dataset/Positive/15001_1.jpg  
  inflating: dataset/Positive/15002_1.jpg  
  inflating: dataset/Positive/15003_1.jpg  
  inflating: dataset/Positive/15004_1.jpg  
  inflating: dataset/Positive/15005_1.jpg  
  inflating: dataset/Positive/15006_1.jpg  
  inflating: dataset/Positive/15007_1.jpg  
  inflating: dataset/Positive/15008_1.jpg  
  inflating: dataset/Positive/15009_1.jpg  
  inflating: dataset/Positive/15010_1.jpg  
  inflating: dataset/Positive/15011_1.jpg  
  inflating: dataset/Positive/15012_1.jpg  
  inflating: dataset/Positive/15013_1.jpg  
  inflating: dataset/Positive/15014_1.jpg  
  inflating: dataset/Positive/15015_1.jpg  
  inflating: dataset/Positive/15016_1.jpg  
  inflating: dataset/Positive/15017_1.jpg  
  inflating: dataset/Positive/15018_1.jpg  
  inflating: dataset/Positive/15019_1.jpg  
  inflating: dataset/Positive/15020_1.jpg  
  inflating: dataset/Positive/15021_1.jpg  
  inflating: dataset/Posi

# Split data into train - test

In [14]:
import os
import numpy as np
import shutil

In [15]:
# # Creating Train / Val / Test folders (One time use)
root_dir = 'dataset'
posCls = '/Positive'
negCls = '/Negative'

os.makedirs(root_dir +'/train' + posCls)
os.makedirs(root_dir +'/train' + negCls)
os.makedirs(root_dir +'/val' + posCls)
os.makedirs(root_dir +'/val' + negCls)
os.makedirs(root_dir +'/test' + posCls)
os.makedirs(root_dir +'/test' + negCls)

In [16]:
# Creating partitions of the data after shuffeling
def loadDataset(currentCls):
  src = "dataset"+currentCls # Folder to copy images from

  train_ratio = 0.7
  val_ratio = 0.15
  test_ratio = 0.15

  allFileNames = os.listdir(src)
  np.random.shuffle(allFileNames)
  train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames),
                                                            [int(len(allFileNames)*(1 - (val_ratio + test_ratio))), 
                                                            int(len(allFileNames)*(1 - val_ratio))])


  train_FileNames = [src+'/'+ name for name in train_FileNames.tolist()]
  val_FileNames = [src+'/' + name for name in val_FileNames.tolist()]
  test_FileNames = [src+'/' + name for name in test_FileNames.tolist()]

  print('Total images: ', len(allFileNames))
  print('Training: ', len(train_FileNames))
  print('Validation: ', len(val_FileNames))
  print('Testing: ', len(test_FileNames))

  # Copy-pasting images
  for name in train_FileNames:
      shutil.copy(name, "dataset/train"+currentCls)

  for name in val_FileNames:
      shutil.copy(name, "dataset/val"+currentCls)

  for name in test_FileNames:
      shutil.copy(name, "dataset/test"+currentCls)

In [17]:
loadDataset(posCls)

Total images:  20000
Training:  14000
Validation:  3000
Testing:  3000


In [18]:
loadDataset(negCls)

Total images:  20000
Training:  14000
Validation:  3000
Testing:  3000


# Load data into DataLoader

In [19]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [20]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, DataLoader
import torch.optim as optim
import torchvision

from torchvision.datasets import ImageFolder

import matplotlib.pyplot as plt

In [21]:
batch_size = 128

transform = torchvision.transforms.Compose([ 
                  torchvision.transforms.ToTensor(),
                  #torchvision.transforms.Normalize((0.1307,), (0.3081,))
])

ds_train = "dataset/train"
ds_val = "dataset/val"
ds_test  = "dataset/test"

ds_train = ImageFolder(root=ds_train, transform=transform)
ds_val = ImageFolder(root=ds_val, transform=transform)
ds_test = ImageFolder(root=ds_test, transform=transform)
                                    
dl_train = DataLoader(ds_train, batch_size=batch_size, shuffle=True)
dl_val   = DataLoader(ds_val, batch_size=batch_size)
dl_test  = DataLoader(ds_test, batch_size=batch_size)

## Sample images

In [22]:
labels_map = {
    0: "Negative",
    1: "Positive",
}

rgb_map = {
    0: "R",
    1: "G",
    2: "B",
}

Define sampleImage()

In [23]:
def sampleImage(imgCount = 1, splitChannel = False, width = 15, height = 15):
  if splitChannel:
    fig, ax = plt.subplots(imgCount,3,figsize=(height,width))
    for j in range(0,imgCount):
      sample_idx = torch.randint(len(ds_train), size=(1,)).item()
      img, label = ds_train[sample_idx]
      imgs = []
      for i in range(0,3):
        imgs.append(img.permute([1,2,0])[:,:,i])
        if imgCount == 1:
          ax[i].imshow(imgs[i], cmap='viridis'), ax[i].set_title(f"{sample_idx} {labels_map[label]} {rgb_map[i%3]} \nmax= {str(imgs[0].max().item())} \nmin= + {str(imgs[0].min().item())}", fontsize=16)
        else:
          ax[j][i].imshow(imgs[i], cmap='viridis'), ax[j][i].set_title(f"{sample_idx} {labels_map[label]} {rgb_map[i%3]} \nmax= {str(imgs[0].max().item())} \nmin= + {str(imgs[0].min().item())}", fontsize=16)
  else:
    nRows = int((imgCount-1)/3+1)
    fig, ax = plt.subplots(nRows,3,figsize=(height,width))
    for i in range(0,nRows*3):
        sample_idx = torch.randint(len(ds_train), size=(1,)).item()
        img, label = ds_train[sample_idx]
        img = img.permute([1,2,0])
        if imgCount <= 3:
          if i >= imgCount:
            ax[i].axis("off")
          else:
            ax[i].imshow(img, cmap='viridis'), ax[i].set_title(f"{sample_idx} {labels_map[label]} RGB \nmax= {str(img.max().item())} \nmin= + {str(img.min().item())}", fontsize=16)
        else:
          if i >= imgCount:
            ax[int(i/3)][i%3].axis("off")
          else:
            ax[int(i/3)][i%3].imshow(img, cmap='viridis'), ax[int(i/3)][i%3].set_title(f"{sample_idx} {labels_map[label]} RGB \nmax= {str(img.max().item())} \nmin= + {str(img.min().item())}", fontsize=16)
          

    # fig, ax = plt.subplots(1,1,figsize=(height,width))
    # ax.imshow(img.permute([1,2,0]))
    # ax.set_title(f"{sample_idx} {labels_map[label]} RGB \nmax= {str(img.max().item())} \nmin= + {str(img.min().item())}", fontsize=16)

Sample images: RGB

> result is best when imgCount is divisible by 3

In [ ]:
sampleImage(3)

Sample images: split channel

In [ ]:
sampleImage(2, splitChannel=True)

# Create model

> parameters are automatically randomized everytime a model is created

Pretrained models

In [24]:
from torchvision.models import resnet50, ResNet50_Weights

In [25]:
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

Custom models

In [26]:
model(torch.zeros(128,3,227,227)).size()

torch.Size([128, 1000])

In [44]:
model = nn.Sequential(
    nn.Conv2d(3, 32, kernel_size=3),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(32, 64, kernel_size=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Conv2d(64, 128, kernel_size=2),
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2),
    nn.Flatten(),
    nn.Dropout(0.5),
    nn.Linear(93312, 128),
    nn.ReLU(),
    nn.Linear(128, 2),
    nn.Sigmoid()
)

model(torch.zeros(128,3,227,227)).size()

torch.Size([128, 2])

In [28]:
params = list(model.named_parameters())
name, param = params[0]
# if param.requires_grad: 
  # print(name, param.data)

In [29]:
# nn.CrossEntropyLoss = nn.LogSoftmax + `nn.NLLLoss
loss_fn = nn.CrossEntropyLoss()  # 
optimizer = optim.Adam( model.parameters(), lr=0.005 )

In [31]:
num_epoch = 10
model = model.to('cuda:0')

for epoch in range(num_epoch):
    trn_loss, val_loss = 0.0, 0.0    
    correct, val_correct = 0, 0

    model.train()    
    for i,(x, y) in enumerate(dl_train):
        x, y = x.to('cuda:0'), y.to('cuda:0')
        #y_onehot = nn.functional.one_hot(y, num_classes=10).float()   

        # nn.CrossEntropyLoss()( y_pred, y )
        # Note that: y can be either one-hot or class index
        
        optimizer.zero_grad()
        y_predict = model(x)        
        loss = loss_fn(y_predict, y)  # nn.CrossEntropyLoss      
        loss.backward()
        optimizer.step()
        
        trn_loss += loss.item()

        pred = y_predict.argmax(dim=1, keepdim=True)          
        correct += pred.eq(y.view_as(pred)).sum().item()
        # print(f'Batch {i}')            
        
    print(f'Epoch {epoch}: Train loss: {trn_loss/len(dl_train):8.5f}, Train acc: {100*correct/len(dl_train.dataset):6.2f}%')            

    model.eval()
    val_correct = 0
    with torch.no_grad():
      for x, y in dl_val:
          x, y = x.to('cuda:0'), y.to('cuda:0')
          #y_onehot = nn.functional.one_hot(y, num_classes=10).float()  

          y_predict = model(x)                    
          loss = loss_fn(y_predict, y)

          val_loss += loss.item()

          pred = y_predict.argmax(dim=1, keepdim=True)  
          val_correct += pred.eq(y.view_as(pred)).sum().item()
          
      print(f'\t Valid loss: {val_loss/len(dl_val):8.5f}, Valid acc: {100*val_correct/len(dl_val.dataset):6.2f}%')            

Epoch 0: Train loss:  0.10147, Train acc:  97.79%
	 Valid loss: 10.58336, Valid acc:  52.95%
Epoch 1: Train loss:  0.02849, Train acc:  99.30%
	 Valid loss:  0.02356, Valid acc:  99.33%
Epoch 2: Train loss:  0.01497, Train acc:  99.55%
	 Valid loss:  0.00977, Valid acc:  99.77%
Epoch 3: Train loss:  0.00907, Train acc:  99.73%
	 Valid loss:  0.01745, Valid acc:  99.63%
Epoch 4: Train loss:  0.00741, Train acc:  99.74%
	 Valid loss:  0.03290, Valid acc:  99.65%


KeyboardInterrupt: ignored